In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.sql import SparkSession
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import StringIndexer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
29,application_1579214001909_0030,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Selected Data

In [4]:
train_df = ss.read.parquet("s3a://usf.msds.stillbikeshare/noutlier_train")
valid_df = ss.read.parquet("s3a://usf.msds.stillbikeshare/noutlier_test")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
va = VectorAssembler(outputCol="features", inputCols=train_df.columns[0:-1])
train_va = va.transform(train_df).select("features", "label")
train_va.cache()

test_va = va.transform(valid_df).select("features", "label")
test_va.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[features: vector, label: int]

In [6]:
f_train, f_valid = train_va.randomSplit([0.8, 0.2])
f_train.cache()
f_valid.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[features: vector, label: int]

In [7]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol="features", maxIter=10, maxBins = 100, maxDepth=5)
gbtmodel = gbt.fit(f_train)
gbtpredicts = gbtmodel.transform(f_valid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(gbtpredicts)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 1252.32

In [9]:
from pyspark.ml.evaluation import RegressionEvaluator
gbt = GBTRegressor(featuresCol="features", maxIter=10, maxBins = 100, maxDepth=10)
gbtmodel = gbt.fit(f_train)
gbtpredicts = gbtmodel.transform(f_valid)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(gbtpredicts)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 1143.79

In [10]:
gbt = GBTRegressor(featuresCol="features", maxIter=10, maxBins = 100, maxDepth=3)
gbtmodel = gbt.fit(f_train)
gbtpredicts = gbtmodel.transform(f_valid)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(gbtpredicts)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 1380.27

In [11]:
gbt = GBTRegressor(featuresCol="features", maxIter=10, maxBins = 100, maxDepth=3)
gbtmodel = gbt.fit(train_va)
gbtpredicts = gbtmodel.transform(test_va)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(gbtpredicts)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 1308.09

# Running Time comparison

In [5]:
import time
start_time = time.time()
from pyspark.ml.evaluation import RegressionEvaluator
gbt = GBTRegressor(featuresCol="features", maxIter=10, maxBins = 100, maxDepth=3)
gbtmodel = gbt.fit(train_va)
gbtpredicts = gbtmodel.transform(test_va)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(gbtpredicts)
print("--- %s seconds ---" % (time.time() - start_time))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--- 50.510488510131836 seconds ---

### Full Data

In [11]:
train_df_full = ss.read.parquet("s3://usf.msds.stillbikeshare/full_train")
valid_df_full = ss.read.parquet("s3://usf.msds.stillbikeshare/full_test")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
va = VectorAssembler(outputCol="features", inputCols=train_df_full.columns[1:])
train_va_full = va.transform(train_df_full).select("features", "label")
train_va_full.cache()

test_va_full = va.transform(valid_df_full).select("features", "label")
test_va_full.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[features: vector, label: int]

In [13]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol="features", maxIter=10, maxBins = 100)
gbtmodel = gbt.fit(train_va_full)
gbtpredicts = gbtmodel.transform(test_va_full)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(gbtpredicts)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 3810.17

In [15]:
paramGrid = ParamGridBuilder().addGrid(gbt.maxIter, [5,10,15]).build()
cv = CrossValidator(estimator=gbt, evaluator=evaluator, numFolds=3, estimatorParamMaps=paramGrid)
cvmodel = cv.fit(train_va_full)
print("Best Max Iter : %s" % cvmodel.bestModel._java_obj.getmaxIter())
print("Accuracy : %s" % RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse").evaluate(cvmodel.bestModel.transform(test_va_full)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o2843.getmaxIter. Trace:
py4j.Py4JException: Method getmaxIter([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)


Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py", line 332, in get_return_value
    format(target_id, ".", name, value))
py4j.protocol